In [1]:
import matplotlib.pyplot as plt
import matplotlib
from notebooks.problems import LinkBudgetProblem, LinkBudgetProblemNonZero, VCMProblem
from PlotUtil import apply_report_formatting, apply_report_formatting_single

# matplotlib.use("pgf")
# matplotlib.rcParams.update({
#     "pgf.texsystem": "xelatex",
#     'font.family': 'sans-serif',
#     'text.usetex': False,
#     'pgf.rcfonts': False
# })

In [2]:
import pandas as pd
from pymoo.algorithms.nsga2 import NSGA2

telesat_passes_df = pd.read_pickle('../Telesat_0_7day.pkl')

N_passes = 20
telesat_passes_df = telesat_passes_df.head(N_passes)

postfix = '_vcm'

In [3]:
# Load modcods
dvbs2_df = pd.read_csv('../modcods/csv/dvbs2.csv', index_col='ID')

In [4]:
import numpy as np

from pymoo.optimize import minimize
from pymoo.util.termination.default import MultiObjectiveDefaultTermination

fc = 30e9
GT_dBk = 13.2
Ptx_dBm = np.arange(20, 40, 0.1)

design_vars = {
    'Ptx_dBm_set': Ptx_dBm,
    'bandwidth_Hz_set': [20e6],
    'Gtx_dBi_set': [20]
}

problem = VCMProblem(telesat_passes_df, dvbs2_df, fc, GT_dBk, design_vars)

sampling, crossover, mutation = problem.generate_default_scm()

termination = MultiObjectiveDefaultTermination(
    #x_tol=1e-8,
    #cv_tol=1e-6,
    f_tol=0.005, #f_tol=0.005,
    nth_gen=5,
    n_last=30,
    n_max_gen=1000, #n_max_gen=1000,
    n_max_evals=100000 #n_max_evals=100000
)

algorithm = NSGA2(
    pop_size=200,
    n_offsprings=500,
    sampling=sampling,
    crossover=crossover,
    mutation=mutation,
    eliminate_duplicates=True,
)

res = minimize(problem,
               algorithm,
               termination=termination,
               seed=1,
               save_history=True,
               verbose=True
               )

print("Done")

n_gen |  n_eval |   cv (min)   |   cv (avg)   |  n_nds  | delta_ideal  | delta_nadir  |   delta_f   
    1 |     200 |  0.00000E+00 |  2.070000000 |       4 |            - |            - |            -
    2 |     700 |  0.00000E+00 |  0.270000000 |       9 |  0.241145397 |  0.519326317 |  0.237526580
    3 |    1200 |  0.00000E+00 |  0.00000E+00 |      13 |  0.124339632 |  1.153858641 |  0.062358198
    4 |    1700 |  0.00000E+00 |  0.00000E+00 |      11 |  0.053236048 |  0.693973136 |  0.076679048
    5 |    2200 |  0.00000E+00 |  0.00000E+00 |      22 |  0.095960272 |  0.570423320 |  0.071594967
    6 |    2700 |  0.00000E+00 |  0.00000E+00 |      26 |  0.004553278 |  0.051416077 |  0.023402687
    7 |    3200 |  0.00000E+00 |  0.00000E+00 |      22 |  0.099730922 |  0.895746558 |  0.037623732
    8 |    3700 |  0.00000E+00 |  0.00000E+00 |      26 |  0.040200650 |  0.359726444 |  0.028523916
    9 |    4200 |  0.00000E+00 |  0.00000E+00 |      33 |  0.000195973 |  0.021585326 |  0.

In [5]:
import pickle
res_dump = res
# res_dump.problem = None
# res_dump.algorithm = None
# res_dump.history = None

f = open("result%s" % postfix, "wb")
pickle.dump(res_dump, f)
f.close()
print('Pickled!')

Pickled!


In [8]:
# Pareto fronts
import numpy as np
import matplotlib.pyplot as plt
from PlotUtil import apply_report_formatting, apply_report_formatting_single
import matplotlib.colors as mcol
import matplotlib.cm as cm

fig = plt.figure(figsize=(3.2, 2.4))

ax = fig.add_subplot(1,1,1)
ax.grid()
ax.set_axisbelow(True)
x = res.F[:,1]
y = ((res.F[:,0] * -1) / 1e9)   # Gigabit

Ptx_dBm_pareto = Ptx_dBm[res.X[:,problem.x_indices['power']].tolist()]
Ptx_dBm_pareto[np.logical_not(res.X[:,problem.x_indices['pass']]).tolist()] = np.nan     

cm1 = mcol.LinearSegmentedColormap.from_list("BlueRed",["c","r"])
min_Ptx = np.nanmin(np.nanmax(Ptx_dBm_pareto, axis=1))
max_Ptx = np.nanmax(np.nanmax(Ptx_dBm_pareto, axis=1))
cnorm = mcol.Normalize(vmin=min_Ptx,vmax=max_Ptx)
cpick = cm.ScalarMappable(norm=cnorm,cmap=cm1)
cpick.set_array([])

cc = cpick.to_rgba(np.nanmax(Ptx_dBm_pareto, axis=1))
ax.scatter(x, y, c=cc, marker='.', s=1)

ax.set_xlabel("Energy used [J]")
ax.set_ylabel("Throughput [Gbit]")
cb = plt.colorbar(cpick,label="Power [dBm]")
cb.ax.tick_params(labelsize=8)
cb.ax.yaxis.label.set_size(8)
apply_report_formatting(4.72, 2.76)
plt.tight_layout()

plt.savefig('D:/git/thesis_report_ae/figures/link_budget_pareto%s.pdf' % postfix)
plt.savefig('D:/git/thesis_report_ae/figures/link_budget_pareto%s.svg' % postfix)
plt.close(fig)

In [7]:
fig = plt.figure(figsize=(3.2, 2.4))

import matplotlib.pyplot as plt
from pymoo.performance_indicator.hv import Hypervolume

# create the performance indicator object with reference point (4,4)
metric = Hypervolume(ref_point=np.array([0,1e9]))

# collect the population in each generation
pop_each_gen = [a.pop for a in res.history]

# receive the population in each generation
obj_and_feasible_each_gen = [pop[pop.get("feasible")[:,0]].get("F") for pop in pop_each_gen]

# calculate for each generation the HV metric
hv = [metric.calc(f) for f in obj_and_feasible_each_gen]

# function evaluations at each snapshot
n_evals = np.array([a.evaluator.n_eval for a in res.history])

# visualze the convergence curve
plt.plot(n_evals, hv, '-o', markersize=3)
plt.title("Convergence")
plt.xlabel("Function Evaluations")
plt.ylabel("Hypervolume")
apply_report_formatting_single()
plt.tight_layout()

plt.savefig('D:/git/thesis_report_ae/figures/hypervolume%s.svg' % postfix)
plt.savefig('D:/git/thesis_report_ae/figures/hypervolume%s.pgf' % postfix)
plt.close()